## Exercise 1
#### What is a perceptron? Be specific.

It is a single-layer neural network used in classification tasks while working with a set of input data. Since perceptron uses classified data points which are already labeled, it is a supervised learning algorithm. This algorithm is used to enable neurons to learn and process elements in the training set one at a time.

## Exercise 2
#### What are the different types of perceptrons? Briefly describe each of them.

There are two types of perceptrons:\
  - Single-layer perceptrons: single-layer perceptrons can learn only linearly separable patterns.\
  - Multi-layer perceptrons: multi-layer perceptrons, also known as feed-forward net-works, contain at least one hidden layer with a non-linear activation function.

## Exercise 3
#### What is a hard margin in a support vector machine model? Be specific.

A hard-margin in a support vector machine model refers to fitting a model with zero errors in the training dataset.

## Exercise 4
#### The effectiveness of a support vector machine model depends on:
(a) kernel\
(b) kernel parameters\
(c) penalty cost parameter\
(d) All of the above
(e) None of the above

### (d) All of the above

## Exercise 5
#### What is/are true about kernel in SVM?
(a) Kernel function map low dimensional data into high dimensional space.\
(b) Kernel function map high dimensional data into low dimensional space.\
(c) It is a similarity function.\
(d) (a) and (c)\
(e) (b) and (c)\
(f) (a), (b) and (c)\
(g) None of the above

### (d) (a) and (c)

In [63]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


## Exercise 6
#### (a) Load the data file to you S3 bucket. Using the pandas library, read the csv data file and create a data-frame called heart.

In [113]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "framingham.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.shape

(4238, 16)

#### (b) Remove observations with missing values.

In [114]:
heart = heart.dropna()
heart = heart.reset_index(drop = True)
heart.shape

(3656, 16)

#### (c) Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and TenYearCHD as the target variable, do the following:
(i) Split the data into train (80%) and test (20%).\
(ii) Using MinMaxScaler, transform all the input variables in the train and test datasets
to 0-1 scale.

• Build a multi-layer perceptron model with one single hidden layer with 4 neurons
(hyperbolic tangent as the activation function) and softmax as the activation function for the output. Use the stochastic descent gradient as the method to estimate
the weights (optimizer = ’sgd’) and metrics = [’accuracy’]. Use epochs =
100 and batch size = 500 to build the model. After that, use the model to predict on the test dataset. Using 15% as the cut-off value, report the recall of this
model.

• Build a multi-layer perceptron model with one single hidden layer with 4 neurons (ReLU as the activation function) and softmax as the activation function for
the output. Use the stochastic descent gradient as the method to estimate the
weights (optimizer = ’sgd’) and metrics = [’accuracy’]. Use epochs = 100
and batch size = 500 to build the model. After that, use the model to predict
on the test dataset. Using 15% as the cut-off value, report the recall of this model.

• Build a support vector machine model using rbf as the kernel. After that, use the
model to predict on the test dataset. Using 15% as the cut-off value, report the
recall of this model.

• Build a support vector machine model using poly as the kernel. After that, use
the model to predict on the test dataset. Using 15% as the cut-off value, report
the recall of this model

In [119]:
# Defining predictor and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

# Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# Transforming the input data to 0-1 scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [120]:
X_train

array([[0.72972973, 0.        , 0.37214137, 0.23145904, 0.39393939],
       [0.16216216, 1.        , 0.31392931, 0.1565681 , 0.31313131],
       [0.67567568, 0.        , 0.24948025, 0.47624818, 0.14141414],
       ...,
       [0.51351351, 0.        , 0.26195426, 0.22976248, 0.46464646],
       [0.24324324, 1.        , 0.21621622, 0.2190984 , 0.28282828],
       [0.10810811, 1.        , 0.10602911, 0.2535143 , 0.36363636]])

In [121]:
md1 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
      tf.keras.layers.Dense(2, activation = 'softmax')
])


md1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
md1.fit(X_train,
        tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
        epochs = 100, 
        batch_size =500, 
        verbose = 0,
        validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))
        
from sklearn.metrics import recall_score
preds = md1.predict(X_test)[:,1]
preds = np.where(preds < 0.15, 0, 1)
recall_score(Y_test, preds)

0.11304347826086956

In [122]:
md2 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 5, activation = 'relu'),
      tf.keras.layers.Dense(2, activation = 'softmax')
])

md2.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
md2.fit(X_train,
        tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
        epochs = 100, 
        batch_size =500, 
        verbose = 0,
        validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))
        

preds_2 = md2.predict(X_test)[:,1]
preds_2 = np.where(preds_2 < 0.15, 0, 1)
recall_score(Y_test, preds_2)

1.0

In [123]:
# Building the svm with kernel = 'rbf'
from sklearn.svm import SVR, SVC

# Building the svm with kernel = 'rbf'
md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

# Predicting on the test dataset
preds_3 = md3.predict_proba(X_test)[:,1]
preds_3 = np.where(preds_3 < 0.15, 0, 1)
recall_score(Y_test, preds_3)

0.6434782608695652

In [124]:
# Building the svm with kernel = 'poly'
md4 = SVC(kernel = 'poly', probability = True).fit(X_train, Y_train)

# Predicting on the test dataset
preds_4 = md4.predict_proba(X_test)[:,1]
preds_4 = np.where(preds_4 < 0.15, 0, 1)
recall_score(Y_test, preds_4)

0.5043478260869565

In [ ]:
md1_recall = list()
md2_recall = list()
md3_recall = list()
md4_recall = list()

for i in range(0,100):
    # Defining predictor and target variables
    X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
    Y = heart['TenYearCHD']

    # Splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

    # Transforming the input data to 0-1 scale
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    # Model 1
    md1 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
      tf.keras.layers.Dense(2, activation = 'softmax')
    ])


    md1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    md1.fit(X_train,
        tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
        epochs = 100, 
        batch_size =500, 
        verbose = 0,
        validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))
        
    from sklearn.metrics import recall_score
    preds = md1.predict(X_test)[:,1]
    preds = np.where(preds < 0.15, 0, 1)
    md1_recall.append(recall_score(Y_test, preds))


     # Model 2
    md2 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 5, activation = 'relu'),
      tf.keras.layers.Dense(2, activation = 'softmax')
    ])

    md2.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    md2.fit(X_train,
        tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
        epochs = 100, 
        batch_size =500, 
        verbose = 0,
        validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))
        

    preds_2 = md2.predict(X_test)[:,1]
    preds_2 = np.where(preds_2 < 0.15, 0, 1)
    md2_recall.append(recall_score(Y_test, preds_2))
    
    # Model 3
    # Building the svm with kernel = 'rbf'
    md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

    # Predicting on the test dataset
    preds_3 = md3.predict_proba(X_test)[:,1]
    preds_3 = np.where(preds_3 < 0.15, 0, 1)
    md3_recall.append(recall_score(Y_test, preds_3))
    
    # Model 4
    # Building the svm with karmel = 'poly'
    md4 = SVC(kernel = 'poly', probability = True).fit(X_train, Y_train)

    # Predicting on the test dataset
    preds_4 = md4.predict_proba(X_test)[:,1]
    preds_4 = np.where(preds_4 < 0.15, 0, 1)
    md4_recall.append(recall_score(Y_test, preds_4))

In [ ]:
# Visualizing model 1
import matplotlib.pyplot as plt

## Plotting iteration results
fig = plt.figure(figsize = (12, 8))

iterations = range(0, 100)
plt.plot(iterations, md1_recall, marker = 'o', color = 'blue',
        label = 'First Model')
plt.plot(iterations, md2_recall, marker = 'o', color = 'orange',
        label = 'Second Model')
plt.plot(iterations, md3_recall, marker = 'o', color = 'red',
         label = 'Third Model')
plt.plot(iterations, md4_recall, marker = 'o', color = 'black',
        label = 'Fourth Model')
plt.xlabel('Number of Iteration')
plt.ylabel('Recall Score')
plt.legend()
plt.grid()
plt.show()

In [ ]:
md1_avg_recall = np.mean(md1_recall)
md2_avg_recall = np.mean(md2_recall)
md3_avg_recall = np.mean(md3_recall)
md4_avg_recall = np.mean(md4_recall)

In [ ]:
print('Average recall score of the model 1: ', round(md1_avg_recall, 2))
print('Average recall score of the model 2: ', round(md2_avg_recall, 2))
print('Average recall score of the model 3: ', round(md3_avg_recall, 2))
print('Average recall score of the model 4: ', round(md4_avg_recall, 2))

The model I would use to predict TenYearCHD is the model 3 because its recall score, in average, is the highest among the four models.